<a href="https://colab.research.google.com/github/lesya7maltseva/pure-Python-/blob/main/%D0%97%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D1%81_Google_Sheets_%D0%B7%D0%B0%D0%BF%D0%B8%D1%81%D1%8C_%D0%B2_%D1%84%D0%B0%D0%B9%D0%BB_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [ ]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2025-01-06 13:26:56--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.112.4
Connecting to gist.github.com (gist.github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2025-01-06 13:26:56--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2025-01-06 13:26:57 (34.4 MB/s) - ‘creds.json’ saved [2

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Как примерно должен выглядеть результат:

In [ ]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [ ]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев** (183 дня).

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023).

**Решение**

Напишите свое решение ниже

In [ ]:
def generate_report(sheet1, sheet2, sheet3):
  import gspread
  from oauth2client.service_account import ServiceAccountCredentials
  from datetime import datetime

  # Указываем необходимые права доступа к таблицам
  scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
          'https://www.googleapis.com/auth/drive']

  # Загружаем ключи аутентификации из файла json
  creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

  # Авторизуемся в Google Sheets API
  client = gspread.authorize(creds)

  #считываем с таблиц построчно
  sheet = client.open("Installments").worksheet("Лист1")
  sheet1_data = sheet.get_all_records()
  sheet2 = client.open("Installments").worksheet("Лист2")
  sheet2_data = sheet2.get_all_records()
  sheet3 = client.open("Installments").worksheet("Лист3")
  sheet3_data = sheet3.get_all_records()

  # Создаем список и в него выбираем должников
  lst_id = []
  last_payment = datetime.strptime("01.03.2023", "%d.%m.%Y").date()
  for i in sheet2_data:
    m = ((last_payment - datetime.strptime(i['expected_payment_date'],"%d.%m.%Y").date()).days // 183) + 1
    if m > 0 :
      i.update({'cnt_payments':m})
      lst_id.append(i)

  # обогащаем список именами студентов
  for id in lst_id:
    for name in sheet1_data:
      if id['student_id'] == name['student_id']:
        id.update({'student_name': name['student_name']})

  # обогащаем список долгом каждого студента
  for id in lst_id:
    for payment in sheet3_data:
      if id['student_id'] == payment['student_id']:
        debt = id['cnt_payments'] * payment['one-time_payment']
        if debt > payment['left_to_pay']:
          id.update({'dedt': payment['left_to_pay']})
        else: id.update({'dedt': debt})

  # записываем в файл
  students_debt = []
  for line in lst_id:
      student = f"Студент {line['student_name']} - долг {line['dedt']} рублей"
      students_debt.append(student)

  student_debt_report = '\n'.join(students_debt)
  with open ('student_debt_report.txt', 'w+') as debts:
    debts.write(student_debt_report)